# I. Importing data and required libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install demoji

In [ ]:
import matplotlib.pyplot as plt
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.utils as ku
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import re
import demoji
import emoji
demoji.download_codes()
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv('/kaggle/input/twitter-airline-sentiment/Tweets.csv')
data

# II. Some EDA(not really)

In [ ]:
virgin_neg, virgin_eq, virgin_pos = (data['airline_sentiment'].loc[data['airline']=='Virgin America'].value_counts())
united_neg, united_eq, united_pos = (data['airline_sentiment'].loc[data['airline']=='United'].value_counts())
southwest_neg, southwest_eq, southwest_pos = (data['airline_sentiment'].loc[data['airline']=='Southwest'].value_counts())
delta_neg, delta_eq, delta_pos = (data['airline_sentiment'].loc[data['airline']=='Delta'].value_counts())
USairways_neg, USairways_eq, USairways_pos = (data['airline_sentiment'].loc[data['airline']=='US Airways'].value_counts())
american_neg, american_eq, american_pos = (data['airline_sentiment'].loc[data['airline']=='American'].value_counts())

print('Sentiment type by airline:\n')
print('Virgin America: positive =',virgin_pos, 'negative =',virgin_neg, 'neutral =', virgin_eq)
print('United: positive =',united_pos, 'negative =',united_neg, 'neutral =', united_eq)  
print('Southwest: positive =',southwest_pos, 'negative =',southwest_neg, 'neutral =', southwest_eq)
print('Delta: positive =',delta_pos, 'negative =',delta_neg, 'neutral =', delta_eq)
print('US Airways: positive =',USairways_pos, 'negative =',USairways_neg, 'neutral =', USairways_eq)
print('American: positive =',american_pos, 'negative =',american_neg, 'neutral =', american_eq)

In [ ]:
plt.xlabel('Airline')
plt.ylabel('Sentiment count')
X = ['Virgin America', 'United', 'Southwest', 'Delta', 'US Airways', 'American']
positive = [virgin_pos, united_pos, southwest_pos, delta_pos, USairways_pos, american_pos]
negative = [virgin_neg, united_neg, southwest_neg, delta_neg, USairways_neg, american_neg]
neutral = [virgin_eq, united_eq, southwest_eq, delta_eq, USairways_eq, american_eq]
br1 = np.arange(6) 
br2 = [x + 0.25 for x in br1] 
br3 = [x + 0.25 for x in br2]

plt.bar(br1, positive, color='b', label='positive', width=0.25)
plt.bar(br2, negative, color='r', label='negative', width=0.25)
plt.bar(br3, neutral, color='g', label='neutral', width=0.25)
plt.xticks([r+0.25 for r in range(6)], X)
plt.title('Types of sentiment count per airline:')
plt.legend()
plt.grid()
plt.show()



# III. Text Preprocessing

In [ ]:
StopWords = set(stopwords.words('english'))

def text_preprocess(text):
    #text = ' '.join([word.lower() for word in text.split() if word.lower() not in StopWords])
    text = ' '.join([word for word in text.split() if ('@' not in word and 'http' not in word and 'https' not in word and '#' not in word)])
    trans = str.maketrans('','',string.punctuation)
    text = text.translate(trans)
    return text

data['text'] = data['text'].apply(text_preprocess)
data.head()        
    

# IV. Tokenization and Lemmatization

In [ ]:
X = data['text']
X = X.tolist()
def lemmatize(data):
    lemmatizer = WordNetLemmatizer()
    lem_data = []
    for text in data:
        lem_text = ''
        for word in text.split():
            word = lemmatizer.lemmatize(word)
            word = lemmatizer.lemmatize(word, pos='v')
            lem_text = lem_text + ' ' + word
        lem_data.append(lem_text)
    return lem_data
        
X_lem = lemmatize(X)
print(X_lem)

In [ ]:
#Handling emojis in data, replacing them with them corresponding text words.
emoji2text = {}
for text in X_lem:
    emoji2text.update(demoji.findall(text))
    
emoji2text

In [ ]:
X_lem_demoji = []
for text in X_lem:
    for emoji, txt in emoji2text.items():
        if emoji in text:
            text = re.sub(emoji, txt+' ', text)
    X_lem_demoji.append(text)
X_lem_demoji

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['airline_sentiment'])
X_train, X_test, y_train, y_test = train_test_split(X_lem_demoji, y, random_state=0)
y_train_true = y_train
y_train = ku.to_categorical(y_train, num_classes=3)
y_test_true = y_test
y_test = ku.to_categorical(y_test, num_classes=3)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
vocab_size = len(tokenizer.word_index)
max_len = 100
train_seq = tokenizer.texts_to_sequences(X_train)
train_pad = pad_sequences(train_seq, maxlen=max_len)
test_seq = tokenizer.texts_to_sequences(X_test)
test_pad = pad_sequences(test_seq, maxlen=max_len)

In [ ]:
def build_model(embedding_dim=300, lr=0.001, nn1=32, nn2=32):
    model = keras.Sequential()
    model.add(keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=max_len))
    model.add(keras.layers.SpatialDropout1D(0.2))
    model.add(keras.layers.Bidirectional(keras.layers.LSTM(nn1, return_sequences=False, dropout=0.2, recurrent_dropout=0.2)))
    #model.add(keras.layers.Bidirectional(keras.layers.LSTM(nn2, dropout=0.3, recurrent_dropout=0.3)))
    #model.add(keras.layers.Dense(embedding_dim, activation='relu'))
    model.add(keras.layers.Dense(3, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=lr), metrics=['accuracy'])
    return model

cv_model = KerasClassifier(build_fn=build_model, epochs=10, batch_size=512)
nn1 = [64,128,256]
#nn2 = [32,64,128]
lr = [0.001,0.01,0.1]
embedding_dim = [300,250,200,150]
batch_size = [64,128,256,512]
param_grid = dict(embedding_dim=embedding_dim, lr=lr, batch_size=batch_size, nn1=nn1)
grid = RandomizedSearchCV(estimator=cv_model, param_distributions=param_grid, n_jobs=1, cv=3)

cv_results = grid.fit(train_pad, y_train)

In [ ]:
cv_results.best_params_

In [ ]:
cv_results.best_estimator_

# V. Training model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
model = keras.Sequential([
    keras.layers.Embedding(vocab_size+1, 150, input_length=max_len),
    keras.layers.SpatialDropout1D(0.2),
    keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)),
    #keras.layers.Bidirectional(keras.layers.LSTM(32, dropout=0.3, recurrent_dropout=0.3)),
    #keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])

opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
my_callbacks = [EarlyStopping(monitor='val_loss', patience=3, mode='min')]
history = model.fit(train_pad, y_train, epochs=10, batch_size=256, validation_data=(test_pad, y_test), callbacks=my_callbacks)

In [ ]:
y_pred = model.predict_classes(test_pad)
y_pred

In [ ]:
#accuracy with Bi-LSTM neural network
print(accuracy_score(y_test_true, y_pred))

In [ ]:
#Tfidf Vectorization, predictions using Logistic Regression.
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5, max_df=0.9)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression(max_iter=1000).fit(X_train_tfidf, y_train_true)
y_pred = clf.predict(X_test_tfidf)
print('Accuracy with Logistic Regression and tfidf Vectorization',accuracy_score(y_test_true, y_pred))